# Funnel Metadynamics tutorial

This tutorial aims to describe how to use scripts that help setup funnel metadynamics (fun-metaD) simulations. The main paper that describe what fun-metaD is [Limogelli *et al* 2013](https://www.pnas.org/content/110/16/6358), but in this tutorial I will describe  [Rhys *et al* 2020](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7467642/) and [Saleh *et al* 2017](https://pubs.acs.org/doi/10.1021/acs.jcim.6b00772) implementation. The main difference between the two implementations is the functional form of the funnel restraints: the original fun-metaD relied on a cone and a cylinder joined to make a funnel using a step function, while the new one uses a single sigmoid function. The Limogelli implementation also requires the protein to be realigned with a reference structure to keep the funnel strictly in place over the binding site, which hurts the performance. The implementation I will describe here allows the funnel to move with the protein.

One of the biggest drawbacks to using fun-metaD for large-scale absolute binding free energy (ABFE) calculations is the difficulty of setting up the simulations. Its hard to know where the funnel should be defined, how big it needs to be, what each of the sigmoid function parameters should be set to, along with the chore of writting PLUMED files, where each protein and ligand system will have sligthly different atom IDs.

The purpose of these scripts is to make this easier, allowing for a quicker setup and much faster automation of large ABFE screening campaigns. I've called the collection of these functions **'funnel_maker'**.

By the end of this tutorial you should know:
1. The basics of what fun-metaD does
2. How to visualise the suggested funnel restraints using PyMOL
3. How to setup PLUMED and OpenMM fun-metaD simulations

Let's get started.

## Part 1

Metadynamics is an enhanced sampling method that biases a simulation along a chosen set of reaction coordinates, or as the MetaD people call them, collective variables (CVs). This bias is deposited at defined time intervals and takes the shape of a Gaussian potential. Investigation of drug binding should involve at least one CV, distance from the drug molecule to the protein, where the distance between them can be biased causing the drug to unbind. However, that single distance is degenerate, meaning many different configurations of the drug in 3D space will not be described by that single distance. It also involves the exploration of a very large volume, hindering convergence. 

Fun-metaD gets around both of these problems, restricting the exploration by using funnel-shaped restraints and reducing degeneracy by using two CVs - 'projection' and 'extent'. See Figure A.

![Figure1](figures/figure1.jpeg)

The restraints that limit in the pp.ext CV follow a sigmoid function:

\begin{equation*}
 S = h \Bigl(\frac{1}{1+e^{b(i-x)}}\Bigr) + f
\end{equation*}

where, S is the maximal distance from the axis, at pp.proj = i, h is the 'wall_width', f is the 'wall_buffer', b is the 'beta_cent' (the steepness at the inflection point), x is the 's_cent' (the inflection point as a value of pp.proj). The exploration along the pp.proj is limited by the 'lower_wall' and 'upper_wall' parameters. The funnel's radius at the narrow end is equal to 'wall_buffer'. 'P0' and 'Px' are the points that define the funnel's vector. From now on I'll refer to them as p0 and p1, respectively.

It should be obvious that there is still some degeneracy - in the plane perpendicular to the projection axis. However, this is good compromise between having sufficient accuracy for describing the binding of a ligand and the tolerable slowdown of using only two CVs.

"Where should the funnel point? How big should it be at the base? Do I need to change the position of the inflexion point? The steepness? How long should the funnel be?"

There aren't any perfect answers to any of these questions. 

Of course, the funnel needs to point out, with the narrow end in the solvent, away from any protein residues. 'Funnel_maker' addresses that question pretty well, most of the vectors it picks for defining the p0 and p1 points are good enough. Its still a good idea to check, by having a look with PyMOL; I will point out one edge case.

As for picking the parameters for the sigmoid - the funnel will need to be smaller than you think. There is usually only one binding site and the funnel should only enclose it, excluding other protein features, with a small 'wall_width'. This really helps with convergence by preventing the drug molecule from exploring irrelevant regions in the free energy surface (FES). Other parameters don't matter that much and the default numbers will suffice in most situations. 

# Part 2

Lets load the relevant Python modules.

In [1]:
import MDAnalysis as mda
import os
import numpy as np
import shutil
import subprocess as sp
import sys
import datetime

The directory with this tutorial contains 'funnel_maker.py', which has all the functions we'll need in this tutorial. To call it from anywhere, add it to your PYTHON_PATH.

In [2]:
from funnel_maker import *

In order to project the funnel, we need to define two points in 3D space, p0 and p1, and get the vector between them. 'Make_funnel' function takes a protein-ligand system and searches the space around the ligand:
1. Defining the p1 point as the center of mass (COM) of protein Calpha atoms within 10A of the ligand.
2. Defining the p0 point as the COM of protein Calpha atoms withing a 20x20x20A grid.

The search is based on very simple, abstract features of a protein-ligand system so it generalises fairly well. I'm sure someone will find exceptions, but the code is flexible enough to have manual inputs given to it, such as defining the center of the search grid using just the XYZ coordinates.

Add 

'load /path/to/current/dir/draw_funnel.py \
load /path/to/current/dir/center_of_mass.py'

to your ~/.pymolrc file, these are the scripts that draws the funnel in PyMOL.

Take a look at the MDM2-nutlin-3a system, simply running the cell below and typing in 'pymol visualise_funnel.pml' in you CLI.

In [3]:
pdb_file = 'input/MDM2-nutlin-3a.pdb' # MDM2 protein with Nutlin 3A

p0, p1 = make_funnel(pdb_file, ligand_name = 'NUT', output_pymol_session=True)

You should see this:

![Figure2](figures/figure2.png)

Simple enough. You might be eager to use the p0 and p1 points (numpy arrays of atom IDs) right away, but that would be a mistake. The PDB provided lacks the hydrogen atoms and the solvent - these things *might* shift the atom IDs and hence you might end up using the wrong atoms in your fun-metaD simulations.

 The correct way to use 'make_funnel' is to provide it a fully prepared system, either a gromacs GRO or Amber RST7/INPCRD + PARM7/PARMTOP files.

In [4]:
gro_file = 'input/3U5J.gro' # BRD4 with a ligand, solvated + ions

p0, p1 = make_funnel(gro_file, ligand_name = 'MOL', output_pymol_session=True)

Load the PyMOL session -  'pymol visualise_funnel.0.pml'

![Figure3](figures/figure3.png)

Great! Below, I'll leave an example of what using Amber files looks like.

In [5]:
struct_file = 'input/2WI2.rst7' # HSP90 protein-ligand system, amber restart file
top_file = 'input/2WI2.prmtop' # HSP90 protein-ligand system, amber parmtop file

p0, p1 = make_funnel(struct_file, top_file, ligand_name = 'MOL', output_pymol_session=True)

Lets investigate an edge case.

In [4]:
structure_file = 'input/2YK9.rst7' # HSP90 protein-ligand system, amber restart file
topology_file = 'input/2YK9.prmtop' # HSP90 protein-ligand system, amber parmtop file

p0, p1 = make_funnel(structure_file, topology_file, output_pymol_session=True)

You should be able to see that the vector is pointing through one of the helices! 

![Figure4](figures/figure4.png)

It probably won't matter too much here, as the radius at the bottom is large enough to let the ligand escape through the side and into the solvent. Nonetheless, lets try using one of the features of funnel_maker, and put in your own coordinates that define the center of the search grid and the position of the p1 point.

In [5]:
p0, p1 = make_funnel(structure_file, topology_file, 
                     grid_coords = [46.263,37.566,37.414], # in angstrom
                     output_pymol_session=True)

The new one looks better. The grid center coordinates used here can be found using PyMOL.
## Part 3.1
Now that we have the p0 and p1 points (the COMs of protein's Calpha atoms), we can put them into a function that writes a template 'plumed.dat' file and, in case you're using an older version of PLUMED, a ProjectionOnAxis.cpp script.

In [7]:
# get all of the ligand and protein atom IDs
p_ids, l_ids = get_protein_ligand_ids(structure_file, topology_file)

# write a plumed file in the current working directory
write_plumed_file(p0, p1, p_ids, l_ids)

And that is all. Now just use that 'plumed.dat' file along with the same exact structure and topology files.

Gromacs run command will look something like:

`gmx_mpi mdrun -deffnm prod -s prod.tpr -plumed plumed.dat -v`
    
While the Amber run parameter file will look something like:
```  10md.in: 300ns NVT production
  &cntrl
    imin = 0, ntx = 5, irest = 1, nstlim = 75000000,
    temp0 = 298.15, ig = -1,
    plumed = 1, plumedfile = 'plumed.dat',
    ntc = 2, ntf = 2, ntt = 3, gamma_ln = 2.0, dt = 0.004,
    ntb = 1,
    ntwx = 25000, ntwe = 0, ntwr = 25000, ntpr = 5000, ioutfm = 1,
    cut = 10.0, iwrap = 1,
    nscm = 1000,
  /```

We can also do the reverse of the last few steps - take a PLUMED file and visualise the funnel using using PyMOL. After you run the cell below, you should see that the last two PyMOL sessions show the same thing, just as expected.

In [6]:
p0, p1, s_cent, beta_cent, wall_width, wall_buffer, lwall, uwall = get_funnel_definitions_from_plumed('plumed.dat')


write_funnel_pymol_session(p0=p0,p1=p1,structure_file=structure_file,topology_file=topology_file,
                           extent=wall_width,extent_buffer=wall_buffer,
                           l_proj=lwall,u_proj=uwall,beta_cent=beta_cent,s_cent=s_cent)

## Part 3.2
Fun-metaD in OpenMM is somewhat different. Since OpenMM has a native metadynamics implementation, we can avaid using PLUMED, reducing the communication overhead between the CPU and the GPU, which dramatically improves performance in hardware setups that are CPU-weak. From my experience fun-metaD runs at about 85% speed of the unbiased simulations.

As of the time of writting (14/10/2020), the fun-metaD in OpenMM is still at the black-box stage, with few tuneable simulation parameters. Later implementation will be far more flexible.

In [8]:
write_run_py(structure_file=structure_file,topology_file=topology_file,
             run_time=10.0,lig_ids=l_ids,p0_ids=p0,p1_ids=p1,lower_wall=lwall,
             upper_wall=uwall,wall_buffer=wall_buffer,wall_width=wall_width,
            s_cent=s_cent,beta_cent=beta_cent)

Now you can simply run the simulation by calling
`python run.py`.
Metadynamics has been implemented from OpenMM version 7.2.0, so fun-metaD won't run on older versions.

If you're interested in what that script does exactly, have a look inside it. You're free to change the parameters inside to suit your simulation needs :)

As for analysing the simulations, have a look at fun-metaD-result_analysis_example.ipynb.